# Introduction to ipyautoui

### Goal in this notebook

In this notebook we will further explore 
[`ipyautoui`](https://github.com/maxfordham/ipyautoui), a Python package that provides a simple way to create interactive user interfaces in Jupyter notebooks. It is built on top of Jupyter widgets and Pydantic and provides a higher-level API for creating interactive widgets.  It is particularly well suited for creating settings panels.

### Steps you will take in this notebook

1. Make a widget from a simple Pydantic model using ipyautoui.
2. Learn some of the differences between an ipyautoui-created widget and one you make yourself.
3. Implement the data selector for the dashboard, suppressing display of unnecessary elements.
4. Address handling constraints for user interface elements.

We begin nearby importing the dashboard for the answer magic.

In [ ]:
import dashboard

In [ ]:
#| default_exp widgets_autoui

## Overview of making a widget with ipyautoui

Most of the work involved in making a widget using ipyautoui is done in making the Pydantic model. The steps are:

1. Define a pydantic model (or write a json schema)
2. Feed the model to AutoUi
3. You get back a widget! With a value! Whose value is easy to save! Or to observe!

### Make a pydantic model

#### Exercise: Write the model

In the cell below, create a pydantic model called `SimpleModel` with one field, called `window_size`, that is an integer.

In [ ]:
# %answer key/03b/01.py

from pydantic import BaseModel

# create model called SimpleModel


### Make the UI

To make a widget from this, start by importing `AutoUi` from `ipyautoui`.

In [ ]:
#| export

from ipyautoui import AutoUi

`AutoUi` takes in the pydantic model and turns it into a widget. Note that it is the class itself, not an instance of the class, that is the argument to `AutoUi`

In [ ]:
ui = AutoUi(SimpleModel)

Run the cell below to display the widget and try changing its value. Note that you can only type numbers (with out decimals) into the box. 

In [ ]:
ui

### Attributes and methods of an AutoUi generated widget

A couple of attributes of the auto-generated widget are particularly useful to know about. 

1. `di_widgets` -- dictionary of widgets, one widget for each pydantic field.
2. `value` -- dictionary of values, one key and value for each pydantic field.

Though one could use `di_widgets` to display the widget for a field outside the auto-generated widget, that is rarely useful. It is sometimes handy, though, to explore the properties of that widget, or to change its properties.

In [ ]:
ui.di_widgets

In this case, `AutoUi` is representing the `int` in our model with an `IntText` widget. That kind of widget only allows integer values to be typed in it.

The `value` for an `AutoUi`-generated widget is a dictionary. The keys are the fields defined in the pydantic model and the values are the current value of that field.

In [ ]:
ui.value

You can set the value, like with any other widget, but make sure that the value is a dictionary.

In [ ]:
ui.value = {"window_size": 11}

The value can also be observed, though there is a twist -- it is `_value` that you observe, not `value`. The observer below simply prints the value of the widget.


In [ ]:
# This observer function just prints to the screen
def print_value(change):
    print(f"{change['new']=}")
    
# NOTE that we are observing changes in _value rather than value
ui.observe(print_value, "_value")

#### Exercise

Change the value of `ui` in the cell below and confirm that you get the message you expect.

In [ ]:
# %answer key/03b/03.py


### Adding constraints to the model can affect the generated widget

Next, let's add constraints to window size, one constraint at a time, to see how they affect the widget that is generated.

In [ ]:
from typing import Annotated
from pydantic import Field

class SimpleModel2(BaseModel):
    window_size : Annotated[int, Field(ge=2)]

In [ ]:
ui2 = AutoUi(SimpleModel2)
ui2

Oh my! Apparently you can set individual fields to values that are invalid. Since we didn't provide a default value, pydantic assumes it was 0, which violates our constraint.

While there is a clear indication of what the error is, it might be nice to set up the interface so that the user cannot enter invalid values.

Let's check to see what kind of widget ipyautoui made for us:

In [ ]:
ui2.di_widgets

`IntText` doesn't incorporate limits so `BoundedIntText` might be a better representation of this field. We can tell ipyautoui to use that widget via a dictionary passed into the `json_schema_extra` argument. The dictionary key that specifies the widget is `autoui`.

In [ ]:
class SimpleModel3(BaseModel):
    window_size : Annotated[
        int, 
        Field(
            ge=2, 
            json_schema_extra=dict(autoui='ipywidgets.BoundedIntText')
        )
    ]

In [ ]:
ui3 = AutoUi(SimpleModel3)
ui3

Nice, now the user cannot set the value below 2! As you can see below, that is because the `min` attribute of the `BoundedIntText` is set to 2.

In [ ]:
ui3.di_widgets

Next we add the constraint that the window size must be less than or equal to 100. We also use the `description` argument to `Field` to add a brief description of the field to the user interface.

In [ ]:
class SimpleModel4(BaseModel):
    window_size: Annotated[int, Field(ge=2, le=100, description="Size of smoothing window")]

In [ ]:
ui4 = AutoUi(SimpleModel4)
ui4

Note that we have also let `ipyautoui` choose the widget -- with both an upper and lower limit provided it makes a slider.

## The data selector

### Selector from the pydantic notebook 

Let's import the pydantic model we finished up the last notebook with. If you get an error here, make sure you ran every cell from the last notebook.

In [ ]:
#| export

from dashboard_pydantic.pydantic_model import DataSelectorModel

Next, we create a UI for the model.

In [ ]:
#| export
controls = AutoUi(DataSelectorModel)

*We recommend that you make a new view for this output so that you can see the effect of changes we make later.*

#### Exercise: make an invalid combination of `window_size` and `polynomial_order`

### Controlling the display of `AutoUi` elements

Many of the user interface elements that `AutoUi` generates can be hidden. The attributes that control the display typically begin with `show_` followed by `iypautoui`'s name for the element.

The image below shows a couple of those elements.

![Image of ipyautoui widget with save button bar and validation labeled](static/savebutton_bar_and_validation.png)

In the cell below we hide both of those elements.

In [ ]:
controls.show_savebuttonbar = False
controls.show_validation = False

Note that even after you hide the details of validation there is still a valid/invalid mark.

#### Exercise

In the cell below make the controls show the validation again.

In [ ]:
# %answer key/03b/05.py


### Enforcing the `window_size`/`polynomial_order` constraints.

It would be nice to set up the user interface so that the user cannot set invalid values.

There are a few options here:

1. Change one or both of the ranges of `window_size` and `polynomial_order`
    1. When window size is being changed and a conflict arises, either
        1. update the maximum value of the polynomial order to be consistent with window size, with the side effect that the value will be updated to.
        1. update the value of `polynomial_order` but not its maximum.
    1. When the polynomial order is changed, either
        1. update the minimum allowed window size, or
        1. update the value of the window size.
1. reset the controls to a valid value and display a warning to the user.

It does not matter so much which of these you choose, but you do need to choose one.

In the first version of the dashboard we did choice 1.A.a -- when `window_size` changed the `polynomial_order` was set to the smaller of 10 and `window_size - 1`.

Here we will implement 2 because it is a little different than what we did in the first round of the dashboard. It also places the responsibility of deciding the correct course of action (increase `window_size` or decrease `polynomial-order`) on the user. 

Our approach is to observe the `_value` of the widget, try to make a valid model out of it, and if that fails set the widget to the old value. `ipyautoui` will take care of displaying an appropriate error message for us.

The observe below is created using a [*closure*](https://en.wikipedia.org/wiki/Closure_(computer_programming)), which is a function that is created inside of another function where the inner function uses some of the variables from the outer function. Here we do that instead of treating `ui` as a global variable.

In [ ]:
#| export

from pydantic import ValidationError

In [ ]:
#| export

def make_enforcer(ui):
    """
    Make a function that can be used to observe changes on a 
    user interface element.

    Parameters
    ----------

    ui: an AutoUi widget

    Returns
    -------

    callable
        A function that can be used as the observer of a traitlets event.
    """
    def constraint_enforcer(change):
        """
        Reset widget to the most recent valid value if the new
        value results in an invalid value.
        """
        try:
            # Every AutoUi widget has a copy of the model class
            # We'll try validating the value in change["new"] and see if it works.
            # If the model is valid, we do not need to do anything. The case where the 
            # model is invalid, which means we need to reset the UI to the last valid
            # settings, is handled in the except clause below.
            ui.model.model_validate(change["new"])
        except ValidationError:
            # ⚠️ This is what resets the value of the widget if the user has entered an
            #    invalid combination of values.
            # That failed, so reset the ui to the old value
            ui.value = change["old"]

    return constraint_enforcer

Run the cell below to add the observer and manipulate the controls to see the effect that the obserer has.

In [ ]:
#| export
controls.observe(make_enforcer(controls), "_value")

## Save the module with nbdev

In [ ]:
from nbdev.export import nb_export

nb_export('03b_ipyautoui.ipynb', 'dashboard_pydantic')